In [2]:
import numpy as np
from scipy.stats import qmc
import matplotlib.pyplot as plt

In [3]:
# atur parameter
theta = np.pi/4
r = 0.95

error_max = 10^(-5)

npoint = 20
dim = 3

iter_max = 100

boundaries = np.array([(-10,10) for _ in range (dim)])

def objective_function(x): # x:tuple n-dimension
    f = 0
    """Schwefel"""
    for i in range (len(x)):
        sum_sq = 0
        for j in range (i+1):
            sum_sq += x[j]
        f += sum_sq**2 
    """2^n Minima"""
    # for i in range (len(x)):
    #     f += (x[i]**4-16*x[i]**2+5*x[i])
    """Rastrigin"""
    # for i in range (len(x)):
    #     f += (x[i]**2-10*np.cos(2*np.pi*x[i])+10)

    return f

In [4]:
"""GENERATE POINTS USING SOBOL SEQUENCE"""
def generate_points(dim: int, npoint: int, low=-10, high=10, sobol=True, seed=None):
  """
  Generates points within the specified bounds.

  Args:
      dim: Number of dimensions.
      npoint: Number of points to generate.
      low: Lower bound for each variable (scalar or list/numpy array).
      high: Upper bound for each variable (scalar or list/numpy array).
      sobol: Flag indicating whether to use Sobol sequence (True) or random sampling (False).

  Returns:
      A numpy array of size (npoint, dim) representing the generated points.
  """

  if type(low) != type(high):
    raise TypeError('The type of "low" and "high" should be the same.')

  # Handle boundaries
  if type(low) == int:
    boundaries = [(low, high) for _ in range(dim)]
  elif type(low) in (list, np.ndarray):
    if len(low) != len(high):
      raise TypeError('The length of "low" and "high" should be the same.')
    else:
      boundaries = [(low[i], high[i]) for i in range(len(low))]

  # Generate points based on the sobol flag
  if sobol:
     sampler = qmc.Sobol(d=dim,scramble=True,seed=seed)
     sample = sampler.random(n=npoint)
     scaled_points = qmc.scale(sample=sample,l_bounds=low,u_bounds=high)

  else:
    # Generate random points
    np.random.seed(seed)
    scaled_points = np.zeros((npoint, dim))
    for i in range(dim):
      min_val, max_val = boundaries[i]
      scaled_points[:, i] = np.random.uniform(min_val, max_val, npoint)

  return scaled_points

def generate_Rij(i,j,dim,theta):
    Rn_ij= np.eye(dim)
    Rn_ij[i-1,i-1] = np.cos(theta)
    Rn_ij[i-1,j-1] = -np.sin(theta)
    Rn_ij[j-1,i-1] = np.sin(theta)
    Rn_ij[j-1,j-1] = np.cos(theta)
    return Rn_ij

def generate_Rn(dim,theta):
    Rn = np.eye(dim)
    for i in range(0,dim):
        for j in range (0,i+1):
            product = np.eye(dim)
            product *= generate_Rij(dim-i-1,dim+1-j-1,dim,theta)
        Rn *= product
    return Rn

In [5]:
def update_point(set_of_points,objective_function,Sn):
    fitness = np.asarray([objective_function(ind) for ind in set_of_points])
    i_g = np.argmin(fitness)
    x_i_g = set_of_points[i_g]

    new_set_of_points = np.copy(set_of_points)
    dim = set_of_points.shape[1]

    for i in range(len(new_set_of_points)):
        poin = np.dot(Sn,set_of_points[i].reshape(-1,1)) - np.dot((Sn-np.identity(dim)),x_i_g.reshape(-1,1))
        new_set_of_points[i] = poin.T
    return new_set_of_points


In [6]:
def iter_error(old_set_of_points,new_set_of_points):
    err = 0
    for i in range (old_set_of_points.shape[0]):
        diff = np.abs(np.linalg.norm(old_set_of_points[i]) - np.linalg.norm(new_set_of_points[i]))
        if diff>err:
            err = diff
    return err

In [7]:
Rn = generate_Rn(dim,theta)
Sn = r*Rn
iter = 0
iter_points = {}
iter_points[iter] = generate_points(dim,10,[-10,-10,-10],[10,10,10],sobol=True)
while iter <= iter_max :
    iter_points[iter+1] = update_point(iter_points[iter],objective_function,Sn)
    error = iter_error(iter_points[iter],iter_points[iter+1])
    if error < error_max:
        break
    iter += 1

return_points = [iter_points[iter][:,i].mean() for i in range(dim)]
return_points

c:\Users\ACER\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_qmc.py:763: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)


[-0.39759781610014533, 1.6052608349420978, -1.9212348477558554]

In [8]:
def spiral_opt(objective_function,boundaries,theta,radius,max_iter,max_error):
    dim = boundaries.shape[0]
    Rn = generate_Rn(dim,theta)
    Sn = radius*Rn
    iter = 0
    iter_points = {}
    iter_points[iter] = generate_points(dim,10,boundaries[:,0],boundaries[:,1],sobol=True)
    while iter <= max_iter :
        iter_points[iter+1] = update_point(iter_points[iter],objective_function,Sn)
        error = iter_error(iter_points[iter],iter_points[iter+1])
        if error < max_error:
            break
        iter += 1

    return_points = np.array([iter_points[iter][:,i].mean() for i in range(dim)])
    return return_points

spiral_opt(objective_function=objective_function,
           boundaries=boundaries,
           theta=theta,
           radius=r,
           max_iter=iter_max,
           max_error=error_max)

array([ 0.09339045, -0.2611575 ,  0.2610552 ])

Auto

In [9]:
# import importlib.util
# import re

# spec = importlib.util.spec_from_file_location("SPO", r"D:\OneDrive - Institut Teknologi Bandung\[AKADEMIK]\Semester 7-8\TA\Thesis\Spiral Optimization\new\Spiropt.py")
# SPO = importlib.util.module_from_spec(spec)
# spec.loader.exec_module(SPO)

import Spiropt as SPO
import importlib
importlib.reload(SPO)

<module 'Spiropt' from 'd:\\OneDrive - Institut Teknologi Bandung\\[AKADEMIK]\\Semester 7-8\\TA\\Thesis\\Spiral Optimization\\new\\Spiropt.py'>

In [13]:
SPO.spiral_opt(objective_function=objective_function,
           boundaries=boundaries,
           n_point=npoint,
           theta=theta,
           radius=r,
           max_iter=iter_max,
           max_error=error_max,
           seed=0)

(array([ 0.02020572,  0.02487532, -0.10038607]), 0.005499218100389501)